In [1]:
import os

In [2]:
%pwd

'c:\\Users\\mayan\\Deep learning\\project2\\1\\Chicken disease classfication\\Chicken-Disease-Classification--Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\mayan\\Deep learning\\project2\\1\\Chicken disease classfication\\Chicken-Disease-Classification--Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
import os
import sys
# Add the src directory to Python path
sys.path.append(os.path.abspath('src'))
# OR if you're inside a notebook folder:
sys.path.append(os.path.abspath('../src'))

In [8]:
from cnnClassifier import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # First download the file
    data_ingestion.download_file()
    # Then extract it
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2024-11-10 23:46:24,471: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-10 23:46:24,475: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-10 23:46:24,477: INFO: common: created directory at: artifacts]
[2024-11-10 23:46:24,479: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-10 23:46:27,254: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11636724
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b7c05d4d2d15b0f33e6ee026963afe641b734162c235ba3910daad9d8146954c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E42A:CB26F:479D2F:5A5032:6730F87B
Accept-Ranges: bytes
Date: Sun, 10 Nov 2024 18:16:29 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10242-MAA
X-Cache: M